# PyData Carolinas VM Assignment Bot

0. Listen for "vm please" requests in https://gitter.im/ibm-et/pydata2016
1. Get a list of all VMs tagged pydata2016 from the SoftLayer (SL) API
2. Filter out all VMs that already have a non-blank note indicating the "owner" of the VM
3. Randomly pick one unassigned VM
4. Fetch the notebook and dashboard server credentials stored in the VM user metadata
5. Store the Gitter username in the VM notes field
6. Respond with links to the services on the VM and credentials
7. Show recent log messages and VM ownership in simple dashboard tables

In [ ]:
import json
import os
import random
import SoftLayer
import collections
from datetime import datetime
from urth.widgets.widget_channels import channel
from tornado.httpclient import AsyncHTTPClient, HTTPClient, HTTPRequest
from tornado.websocket import websocket_connect
from tornado import ioloop

If True, use the parente/sandbox channel and don't save the owner info back to SoftLayer to avoid exhausting the pool. Otherwise, use the ibm-et/pydata2016 channel and write the owner info to the SoftLayer VM notes field.

In [ ]:
DEV = True

Make sure `SOFTLAYER_TOKEN` and `GITTER_TOKEN` are set in the environment. Or set them here.

In [ ]:
SOFTLAYER_TOKEN = os.getenv('SOFTLAYER_TOKEN')
GITTER_TOKEN = os.getenv('GITTER_TOKEN')
TRIGGER = 'vm please'
HEALTH_CHECK_INTERVAL_MS = 10000
CHANNEL_ID = '57766fa9c2f0db084a20f432' if DEV else '5776b21cc2f0db084a20fe67'
HEADERS = {
    'Authorization': 'Bearer ' + GITTER_TOKEN,
    'Content-Type': 'application/json'
}

Globals used across functions and inspected for admin dashboard display.

In [ ]:
recent = collections.deque(maxlen=50)
stream = None

Monitoring functions.

In [ ]:
def log(msg):
    '''
    Put a message in a ring buffer. Put the entire buffer on the log channel
    as status sorted newest to oldest.
    '''
    recent.append((str(datetime.now()), msg))
    channel('log').set('recent', reversed(list(recent)))

In [ ]:
def notify_vm_status():
    '''
    Put the FQDN and its owner on the vms channel as status.
    '''
    channel('vms').set('status', 
                       [(instance['fullyQualifiedDomainName'], instance.get('notes', '')) for instance in instances])

### SoftLayer Client

In [ ]:
sl_client = SoftLayer.create_client_from_env(username='ibmetech-pparente', api_key=SOFTLAYER_TOKEN)

In [ ]:
vm_mgr = SoftLayer.VSManager(sl_client)

In [ ]:
instances = vm_mgr.list_instances(tags=['pydata2016'], mask='id,notes,fullyQualifiedDomainName,userData')
notify_vm_status()

In [ ]:
def assign_vm(username):
    '''
    Assign a virtual machine to the user. If the user already owns a VM,
    return that same VM instead of assigning a new one. 
    
    Raises IndexError if there are no unassigned VMs left to dole out.
    '''
    available = []
    for instance in instances:
        owner = instance.get('notes', '').strip()
        if owner == username:
            instance['config'] = json.loads(instance['userData'][0]['value'])
            return instance
        elif not owner and instance.get('userData', []):
            available.append(instance)
    
    instance = random.choice(available)
    if not DEV:
        # Assign VMs for real if we're not in dev mode
        # Otherwise, only mark them locally
        rv = vm_mgr.edit(instance['id'], notes=username)
    # Update the notes locally too so we don't have to hit the API again
    instance['notes'] = username
    # Make the user data more easily accessible
    instance['config'] = json.loads(instance['userData'][0]['value'])
    notify_vm_status()
    return instance

### Gitter Client

In [ ]:
def on_request_vm(msg):
    '''
    Assign a VM to the user that asked for one and respond with
    the URL and credentials the user needs to access the notebook
    server and dashboard server running on it.
    '''
    c = HTTPClient()
    
    # Get the username
    try:
        username = msg['fromUser']['username']
    except KeyError:
        return log('abort: could not get username from message')

    # Make sure we can respond to the user privately
    log('creating one-to-one chat with {}'.format(username))
    resp = c.fetch('https://api.gitter.im/v1/rooms',
                    raise_error=False,
                    method='POST',
                    headers=HEADERS,
                    body=json.dumps({'uri': username}))
    if resp.code >= 400:
        return log('abort: could not start one-to-one convo with {}'.format(username))
    room = json.loads(resp.body.decode('utf-8'))['id']
    
    # Assign a VM to the user
    try:
        vm = assign_vm(username)
    except IndexError as e:
        log('abort: out of VMs: {}'.format(e))
        msg = 'Oh no! We have no VMs left. Tell an instructor!'
    else:
        msg = '''Hi, {username}. Here's your machine details:
Jupyter Notebook: http://{fullyQualifiedDomainName}:8888 (password: {config[nb_password]})
Jupyter Dashboards: http://{fullyQualifiedDomainName}:3000 (username: pydata, password: {config[db_password]})
'''.format(username=username, **vm)

    # Tell the user the VM details privately
    log('sending VM details to {}'.format(username))
    resp = c.fetch('https://api.gitter.im/v1/rooms/{}/chatMessages'.format(room),
                   raise_error=False,
                   method='POST',
                   headers=HEADERS,
                   body=json.dumps({'text': msg}))
    if resp.code >= 400:
        return log('abort: could not send VM info to {}'.format(username))

    log('finished assigning {} to {}'.format(vm['fullyQualifiedDomainName'], username))

In [ ]:
def on_complete(resp):
    '''
    Remove the global stream object on long-poll disconnect. 
    It acts as a canary for the reconnection logic.
    '''
    global stream
    stream = None
    log('disconnected with code {}'.format(resp.code))

In [ ]:
def on_messages(msgs):
    '''
    Handle one or more messages received in the configured
    Gitter channel. Assign a VM to a message sender if he/she
    says the trigger phrase.
    '''
    for msg in msgs.decode('utf-8').split('\r'):
        msg = msg.strip()
        if not msg: continue        
        log(msg)

        msg = json.loads(msg)
        # Continue processing if the trigger is present
        if TRIGGER in msg['text'].lower():
            try:
                on_request_vm(msg)
            except Exception as e:
                log('abort: exception in on_request_vm: '.format(e))

In [ ]:
def start():
    '''
    Start a long-poll to the Gitter stream API for chat messages
    in the preconfigured channel.
    '''
    global stream
    http_client = AsyncHTTPClient(force_instance=True)
    req = HTTPRequest('https://stream.gitter.im/v1/rooms/{}/chatMessages'.format(CHANNEL_ID), 
                      headers=HEADERS,
                      streaming_callback=on_messages,
                      connect_timeout=2592000,
                      request_timeout=2592000)
    stream = http_client.fetch(req, callback=on_complete)

### Monitoring

In [ ]:
def check_health():
    '''
    Check if the global stream object exists. If not, schedule a 
    new long-poll connection.
    '''
    global stream
    if stream is None:
        log('connecting to stream')
        ioloop.IOLoop.current().call_later(0, start)

Create a periodic callback that checks for long-poll liveliness. Schedule it to run the check every 10 seconds so that we don't stay disconnected for too long.

In [ ]:
try:
    heartbeat.stop()
except NameError:
    pass
heartbeat = ioloop.PeriodicCallback(check_health, HEALTH_CHECK_INTERVAL_MS)

In [ ]:
heartbeat.start()
check_health()

### Monitoring

Shows recent messages and current VM assignment status.

In [ ]:
%%html
<template is="urth-core-bind" channel="log">
    <table>
        <caption>Recent messages</caption>
        <thead>
            <tr>
                <th></th>
                <th>Local time</th>
                <th>Message</th>
            </tr>
        </thead>
        <tbody>
        <template is="dom-repeat" items="[[recent]]">
            <tr>
                <td>[[index]]</td>
                <td>[[item.0]]</td>
                <td>[[item.1]]</td>
            </tr>
        </template>
        </tbody>
</template>

In [ ]:
%%html
<template is="urth-core-bind" channel="vms">
    <table>
        <caption>VM assignments</caption>
        <thead>
            <tr>
                <th></th>
                <th>Host</th>
                <th>Owner</th>
            </tr>
        </thead>
        <tbody>
        <template is="dom-repeat" items="[[status]]">
            <tr>
                <td>[[index]]</td>
                <td>[[item.0]]</td>
                <td>[[item.1]]</td>
            </tr>
        </template>
        </tbody>
</template>